In [1]:
import os

In [2]:
%pwd

'/home/lucifer/Documents/Disease_Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/lucifer/Documents/Disease_Classification'

In [5]:

os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/adarshghimire10/Disease_Classification.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = 'adarshghimire10'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '5a8c90eae9bef971a846e7f1d614aa58370fc271'

In [6]:
import tensorflow as tf

2024-07-20 20:37:39.264721: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-20 20:37:39.265344: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-20 20:37:39.268865: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-20 20:37:39.304772: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-20 20:37:39.980884: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [20]:
model = tf.keras.models.load_model("/home/lucifer/Documents/Disease_Classification/artifacts/training/model.h5")

In [34]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int


In [35]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml,create_directories,save_json

In [36]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)-> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="/home/lucifer/Documents/Disease_Classification/artifacts/training/model.h5",
            training_data="/home/lucifer/Documents/Disease_Classification/artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri= "https://dagshub.com/adarshghimire10/Disease_Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        
        return eval_config

In [37]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [43]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self._valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self._valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0],"accuracy": self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss":self.score[0],"accuracy": self.score[1]}
            )
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model,"model",registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model,"model")


            


In [45]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(config=eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

Found 139 images belonging to 2 classes.


/home/lucifer/Documents/Disease_Classification/kidney/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 1.0000 - loss: 3.7966e-06


2024/07/20 21:58:22 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
/home/lucifer/Documents/Disease_Classification/kidney/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/07/20 21:58:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
